In [2]:
pip install "qiskit[visualization]" qiskit-aer


  Using cached pylatexenc-2.10.tar.gz (162 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136897 sha256=82ee57e711a4b9ea3fb6fb9886283997eae887df633b4e68ccd1f4f3fb6b3fac
  Stored in directory: /home/pv_linux/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pydot]
Note: you may need to restart the kernel to use updated packages.


In [3]:
%time
from math import gcd
from fractions import Fraction
import random
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator


# -------------------------------------------------
# 1. Controlled modular multiplication mod 15
#    This is a hand-crafted circuit valid for:
#    a in {2, 7, 8, 11, 13}
#    and uses 4 qubits to represent numbers 0..14
# -------------------------------------------------
def multiplication_mod_15_gate(a):
    """
    Uncontrolled gate that multiplies the 4-qubit state by 'a' mod 15.
    Only works for a in {2, 7, 8, 11, 13}.
    """
    if a not in [2, 7, 8, 11, 13]:
        raise ValueError("a must be one of {2, 7, 8, 11, 13} for this hardcoded circuit")

    qc = QuantumCircuit(4, name=f"×{a} mod 15")

    # These patterns implement x -> a*x (mod 15)
    # via permutations and bit-flips, specialized to N=15.
    if a in [2, 13]:
        # cyclic shift in one direction
        qc.swap(0, 1)
        qc.swap(1, 2)
        qc.swap(2, 3)

    if a in [7, 8]:
        # cyclic shift in the opposite direction
        qc.swap(2, 3)
        qc.swap(1, 2)
        qc.swap(0, 1)

    if a == 11:
        # another specific permutation
        qc.swap(1, 3)
        qc.swap(0, 2)

    # Some values of 'a' also require X on all target qubits
    if a in [7, 11, 13]:
        for q in range(4):
            qc.x(q)

    return qc.to_gate()


def controlled_amod15(a, power):
    """
    Controlled-U gate that applies (a^power mod 15) to the 4-qubit data register.
    The control is an extra qubit (added later when appending to the main circuit).
    """
    # Build multiplication by a^power as repeated multiplication by a
    base_mult = multiplication_mod_15_gate(a)
    qc = QuantumCircuit(4, name=f"{a}^{power} mod 15")

    for _ in range(power):
        qc.append(base_mult, range(4))

    # Turn into a single gate and then a controlled gate
    gate = qc.to_gate()
    gate.name = f"{a}^{power} mod 15"
    return gate.control(1)   # 1 control qubit


# -------------------------------------------------
# 2. Quantum Fourier Transform (QFT)
# -------------------------------------------------
def qft(num_qubits):
    """Standard QFT on 'num_qubits' qubits."""
    qc = QuantumCircuit(num_qubits, name="QFT")

    for j in range(num_qubits):
        qc.h(j)
        for k in range(j + 1, num_qubits):
            qc.cp(np.pi / (2 ** (k - j)), k, j)

    # Reverse the order of the qubits (bit-reversal)
    for i in range(num_qubits // 2):
        qc.swap(i, num_qubits - i - 1)

    return qc


# -------------------------------------------------
# 3. Build Shor circuit for N = 15
# -------------------------------------------------
def build_shor_15_circuit(a, n_count=8):
    """
    Build the Shor period-finding circuit for N=15 and base 'a'.
    n_count: number of counting qubits.
    Total qubits: n_count (counting) + 4 (data register).
    """
    # n_count counting qubits + 4 for the modular exponentiation register
    qc = QuantumCircuit(n_count + 4, n_count)

    # Step 1: put counting qubits into superposition
    for q in range(n_count):
        qc.h(q)

    # Step 2: prepare the data register in |1> (i.e. state '1' mod 15)
    # Data register is the last 4 qubits
    qc.x(n_count + 3)   # put the least significant data qubit to |1>

    # Step 3: controlled-U^{2^k} for each counting qubit k
    for k in range(n_count):
        # power = 2^k
        power = 2 ** k
        c_U = controlled_amod15(a, power)
        # Control is counting qubit k; targets are the 4 data qubits
        qc.append(c_U, [k] + list(range(n_count, n_count + 4)))

    # Step 4: apply inverse QFT on the counting register
    qft_dag = qft(n_count).inverse()
    qc.append(qft_dag, range(n_count))

    # Step 5: measure counting register
    qc.measure(range(n_count), range(n_count))

    return qc


# -------------------------------------------------
# 4. Run Shor for N = 15 on the Aer simulator
# -------------------------------------------------
def run_shor_15(a=None, n_count=8, shots=2048, plot=False):
    N = 15

    # choose a if not given
    if a is None:
        # allowed 'a' values that are coprime with 15 and supported by our circuit
        candidates = [2, 7, 8, 11, 13]
        a = random.choice(candidates)

    print(f"Trying to factor N = {N} using Shor with base a = {a}")

    if gcd(a, N) != 1:
        print("gcd(a, N) != 1 -> trivial factor:", gcd(a, N))
        return

    qc = build_shor_15_circuit(a, n_count=n_count)

    # simulate
    simulator = AerSimulator()
    qc_t = transpile(qc, simulator)
    result = simulator.run(qc_t, shots=shots).result()
    counts = result.get_counts()

    if plot:
        plot_histogram(counts)

    print("Measurement counts:", counts)

    # pick the most likely outcome
    measured_str = max(counts, key=counts.get)
    measured_value = int(measured_str, 2)
    print(f"Most frequent result: {measured_str} (decimal {measured_value})")

    # convert to phase s / 2^n_count
    phase = measured_value / (2 ** n_count)
    frac = Fraction(phase).limit_denominator(N)
    s, r = frac.numerator, frac.denominator
    print(f"Phase ≈ {s}/{r} -> estimated period r = {r}")

    if r % 2 != 0:
        print("Estimated r is odd; Shor's post-processing fails. Try again with another 'a'.")
        return

    # Classical post-processing: compute gcd(a^{r/2} ± 1, N)
    x = pow(a, r // 2, N)  # a^(r/2) mod N
    p = gcd(x - 1, N)
    q = gcd(x + 1, N)

    print(f"a^(r/2) mod N = {x}")
    print(f"gcd(a^(r/2) - 1, N) = {p}")
    print(f"gcd(a^(r/2) + 1, N) = {q}")

    if p in [1, N] or q in [1, N]:
        print("Got only trivial factors; try again with a different 'a'.")
    else:
        print(f"Non-trivial factors of {N} found: {p} and {q}")


# -------------------------------------------------
# 5. Run it
# -------------------------------------------------
if __name__ == "__main__":
    # Set a fixed 'a' if you want, e.g. a=7
    run_shor_15(a=7, n_count=8, shots=2048, plot=False)


CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 3.58 μs
Trying to factor N = 15 using Shor with base a = 7
Measurement counts: {'10110101': 1, '10110010': 1, '11011011': 1, '01100100': 1, '01100101': 1, '01000110': 1, '01010100': 2, '10000111': 2, '01111101': 1, '11011100': 2, '01010000': 5, '01000010': 1, '11100010': 1, '10101000': 2, '11101111': 3, '10101110': 2, '01100000': 29, '01100111': 3, '01011011': 1, '01011000': 1, '11000000': 53, '10100000': 20, '11100100': 1, '11001111': 1, '01001000': 5, '10111011': 1, '10011111': 8, '10000000': 402, '01000100': 2, '01101111': 8, '11000100': 1, '01011111': 17, '10011000': 1, '01000000': 186, '10001111': 1, '01110011': 1, '10110000': 6, '01111111': 208, '01010111': 2, '11111111': 423, '11110011': 1, '11010000': 4, '01001111': 3, '11011111': 17, '01101000': 1, '01110000': 2, '11010111': 1, '11101000': 2, '00000000': 534, '10010000': 3, '11010011': 1, '01110111': 2, '10111111': 53, '01101100': 1, '10101111': 5, '11100000': 7, '1000010